In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib
import nltk
nltk.download('all')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize as wt
from nltk.stem import WordNetLemmatizer as wnl
import pyarrow as pa
from pyarrow import Table
import pyarrow.parquet as pq

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/jonasaray/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/jonasaray/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/jonasaray/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/jonasaray/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /home/jonasaray/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloadi

In [21]:
with open('../Dataset/users_items.json/australian_users_items.json', 'r',encoding='utf-8') as file:
    rows = [literal_eval(row) for row in file.readlines()]

user_items = pd.DataFrame(rows)
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [22]:
#user_items = user_items.drop(columns=['user_url'])
user_items = user_items[user_items['items_count'] != 0]
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88303,76561198320136420,321,76561198320136420,http://steamcommunity.com/profiles/76561198320...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88304,ArkPlays7,4,76561198321542621,http://steamcommunity.com/id/ArkPlays7,"[{'item_id': '218230', 'item_name': 'PlanetSid..."
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."


In [23]:
user_items = user_items.drop_duplicates(subset=['steam_id', 'user_id'])
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88303,76561198320136420,321,76561198320136420,http://steamcommunity.com/profiles/76561198320...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88304,ArkPlays7,4,76561198321542621,http://steamcommunity.com/id/ArkPlays7,"[{'item_id': '218230', 'item_name': 'PlanetSid..."
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."


In [24]:
user_items = user_items[user_items['user_id'] != user_items['steam_id']]
user_items

,user_id,items_count,steam_id,user_url,items
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,371,76561198004744620,http://steamcommunity.com/id/MinxIsBetterThanP...,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
...,...,...,...,...,...
88274,Soppro,5,76561198303816325,http://steamcommunity.com/id/Soppro,"[{'item_id': '214490', 'item_name': 'Alien: Is..."
88278,tacopiglet,3,76561198306909488,http://steamcommunity.com/id/tacopiglet,"[{'item_id': '550', 'item_name': 'Left 4 Dead ..."
88281,Ghoustik,4,76561198308105615,http://steamcommunity.com/id/Ghoustik,"[{'item_id': '302830', 'item_name': 'BLOCKADE ..."
88292,POMFP0MF,1,76561198313471778,http://steamcommunity.com/id/POMFP0MF,"[{'item_id': '730', 'item_name': 'Counter-Stri..."


In [25]:
items_for_all_users = pd.DataFrame(columns=['item_id', 'item_name','playtime_forever','playtime_2weeks','user_id'])
for u in user_items['user_id']:
    usuario = user_items[user_items['user_id'] == u];
    df_aux = pd.DataFrame(columns=['item_id', 'item_name','playtime_forever','playtime_2weeks'])
    for ui in usuario['items']:
        aux = pd.DataFrame(ui, columns=items_for_all_users.columns)
        df_aux = pd.concat([aux, df_aux],ignore_index=True)
        df_aux['user_id'] = u
    items_for_all_users = pd.concat([df_aux, items_for_all_users], ignore_index=True)
items_for_all_users = items_for_all_users.sort_values(by='playtime_forever',ascending=False)
items_for_all_users

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
2553295,4000,Garry's Mod,642773,7299,wolop
1411006,212200,Mabinogi,635295,0,Evilutional
1798391,72200,Universe Sandbox,600068,0,tsunamitad
2773537,4000,Garry's Mod,551719,0,jimmynoe
1738295,8500,EVE Online,530882,0,shinomegami
...,...,...,...,...,...
1378863,370300,8-Bit Adventures: The Forgotten Journey Remast...,0,0,FubDuck
1378862,369060,Amygdala,0,0,FubDuck
1378861,366844,Call of Duty: Black Ops III - Awakening DLC Pack,0,0,FubDuck
1378859,365720,Skyrim Script Extender (SKSE),0,0,FubDuck


In [26]:
items_for_all_users['playtime_forever'] = items_for_all_users['playtime_forever'].apply(lambda x : round(x/24,2))
items_for_all_users

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
2553295,4000,Garry's Mod,26782.21,7299,wolop
1411006,212200,Mabinogi,26470.62,0,Evilutional
1798391,72200,Universe Sandbox,25002.83,0,tsunamitad
2773537,4000,Garry's Mod,22988.29,0,jimmynoe
1738295,8500,EVE Online,22120.08,0,shinomegami
...,...,...,...,...,...
1378863,370300,8-Bit Adventures: The Forgotten Journey Remast...,0.00,0,FubDuck
1378862,369060,Amygdala,0.00,0,FubDuck
1378861,366844,Call of Duty: Black Ops III - Awakening DLC Pack,0.00,0,FubDuck
1378859,365720,Skyrim Script Extender (SKSE),0.00,0,FubDuck


In [27]:
items_for_all_users['playtime_2weeks'] = items_for_all_users['playtime_2weeks'].apply(lambda x : round(x/24,2))
items_for_all_users

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
2553295,4000,Garry's Mod,26782.21,304.12,wolop
1411006,212200,Mabinogi,26470.62,0.00,Evilutional
1798391,72200,Universe Sandbox,25002.83,0.00,tsunamitad
2773537,4000,Garry's Mod,22988.29,0.00,jimmynoe
1738295,8500,EVE Online,22120.08,0.00,shinomegami
...,...,...,...,...,...
1378863,370300,8-Bit Adventures: The Forgotten Journey Remast...,0.00,0.00,FubDuck
1378862,369060,Amygdala,0.00,0.00,FubDuck
1378861,366844,Call of Duty: Black Ops III - Awakening DLC Pack,0.00,0.00,FubDuck
1378859,365720,Skyrim Script Extender (SKSE),0.00,0.00,FubDuck


In [28]:
items_for_all_users = items_for_all_users[items_for_all_users['playtime_forever'] != 0.00]
items_for_all_users = items_for_all_users[items_for_all_users['playtime_2weeks'] < items_for_all_users['playtime_forever']]
items_for_all_users

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
2553295,4000,Garry's Mod,26782.21,304.12,wolop
1411006,212200,Mabinogi,26470.62,0.00,Evilutional
1798391,72200,Universe Sandbox,25002.83,0.00,tsunamitad
2773537,4000,Garry's Mod,22988.29,0.00,jimmynoe
1738295,8500,EVE Online,22120.08,0.00,shinomegami
...,...,...,...,...,...
1514715,268850,EVGA Precision XOC,0.04,0.00,CoconutParty
1724944,21110,F.E.A.R.: Extraction Point,0.04,0.00,maddoxx789
1114859,260690,Dark Fall 1: The Journal,0.04,0.00,Midiri91
2333866,34270,SEGA Mega Drive & Genesis Classics,0.04,0.00,federige


In [29]:
games = pd.read_json('../Dataset/steam_games.json/output_steam_games.json', lines=True)
games = games.drop(columns=['url','reviews_url','specs', 'tags','early_access']) # Una vez analizado, se determinó que estas columnas no contienen información relevante
games['año'] = games['release_date'].str.extract(r'(\d{4})')                              # Se toma el dato sobre la fecha en que salió el juego, la cual será relevante más adelante
games = games.drop(columns=['release_date'])                                              # Este campo ya no tiene ninguna relevancia
games = games.dropna(axis=0,how="any",subset=['genres','developer','año'])                # Se eliminan los campos que estén vacios, no puede existir juego si no tiene desarrollador, genero o año de salida
games = games.sort_values(by='id')                                                        # Se ordenan los datos según si id
games = games.drop_duplicates(subset=['title','app_name','año'])                          # Se busca eliminar datos repetidos, pueden existir juegos con el mismo nombre pero que hayan salido en diferentes años o teniendo generos diferentes
games = games.drop(columns=['app_name'])                                                  # Columna con datos duplicados

def floatToStr(n):
    '''Funcion que me permite usar un tipo de dato diferente para los id, el cual mejora su uso para comparaciones.'''
    return str(int(n))

games['id'] = games['id'].apply(floatToStr)                                               # Se usa la función y se retorna a su respectiva serie
games.rename({'id':'item_id'})
games # Mostrar los datos con los que se va a trabajar

,publisher,genres,title,price,id,developer,año
120416,Valve,[Action],Counter-Strike,9.99,10,Valve,2000
120413,Valve,[Action],Team Fortress Classic,4.99,20,Valve,1999
120424,Valve,[Action],Day of Defeat,4.99,30,Valve,2003
120418,Valve,[Action],Deathmatch Classic,4.99,40,Valve,2001
120414,Valve,[Action],Half-Life: Opposing Force,4.99,50,Gearbox Software,1999
...,...,...,...,...,...,...,...
118903,Ubisoft,[Action],Tom Clancy's Ghost Recon Future Soldier - Seas...,24.99,2028055,"Ubisoft Paris,Red Storm Entertainment",2012
118999,Team17 Digital Ltd,[Strategy],Worms Revolution Season Pass,14.99,2028056,Team17 Digital Ltd.,2012
118632,Activision,[Action],Call of Duty®: Black Ops II Season Pass,49.99,2028062,Treyarch,2013
118834,Ubisoft,"[Action, Adventure]",Assassin’s Creed® III Season Pass,29.99,2028103,Ubisoft Montreal,2012


In [32]:
games['price'].unique()

array([9.99, 4.99, 19.99, None, 7.99, 'Free to Play', 'Free', 14.99, 6.99,
       2.99, 15.99, 29.99, 'Third-party', 39.99, 12.99, 14.95, 11.99,
       3.99, 5.99, 7.49, 8.99, 1.99, 54.99, 2.49, 0.99, 24.99, 1.25,
       61.99, 3.49, 32.99, 16.99, 4.49, 15.0, 99.99, 13.99,
       1.5899999999999999, 59.99, 'Free To Play', 1.49, 'Play for Free!',
       17.99, 160.91, 34.99, 5.49, 'Free Demo', 59.95, 49.99, 69.99,
       79.99, 19.98, 'Free to Use', 'Free HITMAN™ Holiday Pack', 13.98,
       44.99, 26.99, 13.37, 18.99, 23.99, 'Play WARMACHINE: Tactics Demo',
       0.49, 149.99, 199.99, 299.99, 'Install Now', 29.96, 5.65, 124.99,
       10.99, 189.96, 2.0, 771.71, 0.98, 9.98, 16.06, 109.99, 64.99,
       89.99, 21.99, 9.95, 'Free Mod', 129.99, 27.99, 12.0, 139.92, 20.99,
       'Play Now', 119.99, 10.0, 4.29, 23.96, 6.49, 1.0, 12.89, 74.99,
       31.99, 289.99, 'Free to Try', 'Install Theme', 41.99, 0.89, 10.93,
       1.29, 1.87, 49.0, 99.0, 199.0, 0.5, 42.99, 3.0, 87.94, 20.0, 7.0,
 

In [34]:
free_games = games[games['price'] == ['Free To Play', 'Free to Play', 'Free']]
free_games

ValueError: ('Lengths must match to compare', (28554,), (3,))

In [30]:
games['genres'] = games['genres'].apply(lambda x : ', '.join(x) if isinstance(x,list) else x)
games['genres'] = games['genres'].str.split(', ')
games = games.explode('genres')
genre = games['genres'].nunique()
genre

22

In [11]:
games['genres'].join(genre)

AttributeError: 'Series' object has no attribute 'join'

In [32]:
genre = games['genres'].unique()
genre

p = pd.DataFrame(['Action', 'Free to Play'], columns=['genres'])
p

,genres
0,Action
1,Free to Play


In [20]:
df_join = games.merge(p,'inner','genres')
df_join

,publisher,genres,title,price,id,developer,año
0,Valve,Action,Counter-Strike,9.99,10,Valve,2000
1,Valve,Action,Team Fortress Classic,4.99,20,Valve,1999
2,Valve,Action,Day of Defeat,4.99,30,Valve,2003
3,Valve,Action,Deathmatch Classic,4.99,40,Valve,2001
4,Valve,Action,Half-Life: Opposing Force,4.99,50,Gearbox Software,1999
...,...,...,...,...,...,...,...
13170,Deep Silver,Action,Saints Row: The Third Season Pass DLC Pack,4.99,901805,Volition,2011
13171,Ubisoft,Action,Tom Clancy's Ghost Recon Future Soldier - Seas...,24.99,2028055,"Ubisoft Paris,Red Storm Entertainment",2012
13172,Activision,Action,Call of Duty®: Black Ops II Season Pass,49.99,2028062,Treyarch,2013
13173,Ubisoft,Action,Assassin’s Creed® III Season Pass,29.99,2028103,Ubisoft Montreal,2012


In [34]:
games = games.rename(columns={"id":"item_id"})
games

,publisher,genres,title,price,item_id,developer,año
120416,Valve,Action,Counter-Strike,9.99,10,Valve,2000
120413,Valve,Action,Team Fortress Classic,4.99,20,Valve,1999
120424,Valve,Action,Day of Defeat,4.99,30,Valve,2003
120418,Valve,Action,Deathmatch Classic,4.99,40,Valve,2001
120414,Valve,Action,Half-Life: Opposing Force,4.99,50,Gearbox Software,1999
...,...,...,...,...,...,...,...
118999,Team17 Digital Ltd,Strategy,Worms Revolution Season Pass,14.99,2028056,Team17 Digital Ltd.,2012
118632,Activision,Action,Call of Duty®: Black Ops II Season Pass,49.99,2028062,Treyarch,2013
118834,Ubisoft,Action,Assassin’s Creed® III Season Pass,29.99,2028103,Ubisoft Montreal,2012
118834,Ubisoft,Adventure,Assassin’s Creed® III Season Pass,29.99,2028103,Ubisoft Montreal,2012


In [35]:
item_for_genre = pd.DataFrame(columns=['item_id', 'item_name','playtime_forever','playtime_2weeks','user_id'])
for g in genre:
    aux_games = games[games['genres'] == g]
    aux = items_for_all_users.merge(aux_games,'inner','item_id')
    item_for_genre = pd.concat([aux, item_for_genre], ignore_index=True)
users = item_for_genre['user_id']
users

/tmp/ipykernel_4315/2026759299.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  item_for_genre = pd.concat([aux, item_for_genre], ignore_index=True)


0                  Forbieden
1                  Adrian_31
2                   SeedyDog
3          TFWUMastab82Sanic
4             Taariqamozilla
                 ...        
3643170        LucarioSakaki
3643171              trawlin
3643172          therealtiga
3643173             federige
3643174           jylesjyles
Name: user_id, Length: 3643175, dtype: object

In [36]:
item_for_genre

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,publisher,genres,title,price,developer,año
0,227240,Construct 2 Free,1084.79,9.38,Forbieden,Scirra,Education,Construct 2,Free to Use,Scirra,2013
1,227240,Construct 2 Free,350.17,0.00,Adrian_31,Scirra,Education,Construct 2,Free to Use,Scirra,2013
2,227240,Construct 2 Free,299.75,0.00,SeedyDog,Scirra,Education,Construct 2,Free to Use,Scirra,2013
3,248170,Clickteam Fusion 2.5,288.29,0.00,TFWUMastab82Sanic,Clickteam,Education,Clickteam Fusion 2.5,99.99,Clickteam,2013
4,227240,Construct 2 Free,223.58,0.00,Taariqamozilla,Scirra,Education,Construct 2,Free to Use,Scirra,2013
...,...,...,...,...,...,...,...,...,...,...,...
3643170,2620,Call of Duty,0.04,0.00,LucarioSakaki,Activision,Action,Call of Duty®,19.99,Infinity Ward,2003
3643171,50300,Spec Ops: The Line,0.04,0.00,trawlin,"2K Games, Missing Link Games",Action,Spec Ops: The Line,29.99,Yager Development,2012
3643172,220,Half-Life 2,0.04,0.00,therealtiga,Valve,Action,Half-Life 2,9.99,Valve,2004
3643173,34270,SEGA Mega Drive & Genesis Classics,0.04,0.00,federige,SEGA,Action,SEGA Mega Drive and Genesis Classics,61.99,D3T Limited,2010


In [51]:
item_for_genre['user_id'].nunique()

27220

In [43]:
time_per_user = []
users_prueba = pd.DataFrame(item_for_genre['user_id'].unique())
for g in genre:
    prueba = games[games['genres']==g]
    for u in users_prueba:
        up = item_for_genre[item_for_genre['user_id'] == u]
        time_per_user.append({"Tiempo":up['playtime_forever'].sum(), "Usuario":u})
print(max(time_per_user, key=lambda t:t['Tiempo']))

{'Tiempo': 99831.12999999999, 'Usuario': 'REBAS_AS_F-T'}


In [42]:
print(max(time_per_user, key=lambda t:t['Tiempo']))

{'Tiempo': 99831.13000000009, 'Usuario': 'REBAS_AS_F-T'}


In [50]:
prueba

Index([1254815, 1254816, 1254817, 1254818, 1254819, 1254820, 1254821, 1254822,
       1254823, 1254824,
       ...
       1946651, 1946652, 1946653, 1946654, 1946655, 1946656, 1946657, 1946658,
       1946659, 1946660],
      dtype='int64', length=691846)

In [98]:
prueba = item_for_genre
prueba2 = prueba.groupby(by=['user_id','genres','año'],)['playtime_forever'].sum()
prueba2.idxmax()

('REBAS_AS_F-T', 'Indie', '2016')

In [100]:
prueba2.max()

33968.42

In [102]:
salida = pd.DataFrame(prueba2)
salida = salida[salida['playtime_forever'] < 8760.00]
salida

playtime_forever
user_id    genres   año                   
--000--    Action   2009            222.05
                    2010              0.92
                    2011            271.74
                    2012           4556.08
                    2013             15.12
...                                    ...
zzzmidmiss Sports   2010              8.08
                    2014              0.67
           Strategy 2010              0.42
                    2011              2.88
                    2012             38.46

[1378430 rows x 1 columns]

In [104]:
salida.idxmax()

playtime_forever    (Cow666, Strategy, 2010)
dtype: object

In [105]:
salida.reset_index(inplace=True)
salida

,user_id,genres,año,playtime_forever
0,--000--,Action,2009,222.05
1,--000--,Action,2010,0.92
2,--000--,Action,2011,271.74
3,--000--,Action,2012,4556.08
4,--000--,Action,2013,15.12
...,...,...,...,...
1378425,zzzmidmiss,Sports,2010,8.08
1378426,zzzmidmiss,Sports,2014,0.67
1378427,zzzmidmiss,Strategy,2010,0.42
1378428,zzzmidmiss,Strategy,2011,2.88


In [106]:
salida[salida['genres'] == 'Action'].max()

user_id             zzzmidmiss
genres                  Action
año                       2018
playtime_forever       8743.97
dtype: object

In [109]:
verificacion = salida.groupby(by=['user_id', 'año'])['playtime_forever'].sum() 
df_ver = pd.DataFrame(verificacion)
df_ver.reset_index(inplace=True)
df_ver = df_ver[df_ver['playtime_forever'] < 8760.00]
df_ver

,user_id,año,playtime_forever
0,--000--,2006,77.08
1,--000--,2009,222.05
2,--000--,2010,2.76
3,--000--,2011,648.47
4,--000--,2012,4930.25
...,...,...,...
307287,zzzmidmiss,2010,51.56
307288,zzzmidmiss,2011,104.77
307289,zzzmidmiss,2012,602.78
307290,zzzmidmiss,2013,17.65


In [111]:
df_ver

27218

In [113]:
salida

27220

In [118]:
prueba_join = salida.merge(df_ver['user_id'], how='inner')
prueba_join.drop_duplicates(inplace=True)
prueba_join

,user_id,genres,año,playtime_forever
0,--000--,Action,2009,222.05
10,--000--,Action,2010,0.92
20,--000--,Action,2011,271.74
30,--000--,Action,2012,4556.08
40,--000--,Action,2013,15.12
...,...,...,...,...
18687006,zzzmidmiss,Sports,2010,8.08
18687012,zzzmidmiss,Sports,2014,0.67
18687018,zzzmidmiss,Strategy,2010,0.42
18687024,zzzmidmiss,Strategy,2011,2.88


In [119]:
salida2 = salida[salida['genres'] == 'Action']
s3 = salida2.groupby(by=['user_id'])['playtime_forever'].sum()
s3.idxmax()

'Sp3ctre'

### Finalmente

Guardar los valores de la variable salida en un archivo .parquet por medio de la librería pyarrow

In [120]:
# Pyarrow
UserForGenreSchema = pa.schema([('user_id', pa.string()), ('genres', pa.string()), ('año', pa.string()), ('playtime_forever', pa.float32())])
TableUFGS = Table.from_pandas(salida, schema=UserForGenreSchema)
pq.write_table(TableUFGS, '../DatasetFinal/UserForGenre.parquet')

In [122]:
archivo = pd.read_parquet('../DatasetFinal/UserForGenre.parquet')
lkj = archivo[archivo['genres'] == 'Action']
lkj = lkj[lkj['user_id'] == 'Sp3ctre']
lkj.drop(columns=['user_id','genres'], inplace=True)
lkj['playtime_forever'].apply(lambda x : round(x,2))

,año,playtime_forever
527687,1995,9.040000
527688,1999,1.830000
527689,2000,2943.500000
527690,2001,0.540000
527691,2002,9.920000
527692,2003,319.709991
527693,2004,5308.790039
527694,2005,889.119995
527695,2006,37.320000
527696,2007,4699.330078


In [124]:
lkj['playtime_forever'] = lkj['playtime_forever'].apply(lambda x : round(x,0))
lkj

,año,playtime_forever
527687,1995,9.0
527688,1999,2.0
527689,2000,2944.0
527690,2001,1.0
527691,2002,10.0
527692,2003,320.0
527693,2004,5309.0
527694,2005,889.0
527695,2006,37.0
527696,2007,4699.0


In [127]:
lkj.set_index('año',inplace=True)
lkj.to_dict()

KeyError: "None of ['año'] are in the columns"

In [131]:
diccionario = lkj.to_dict()

In [134]:
diccionario['playtime_forever']

{'1995': 9.0,
 '1999': 2.0,
 '2000': 2944.0,
 '2001': 1.0,
 '2002': 10.0,
 '2003': 320.0,
 '2004': 5309.0,
 '2005': 889.0,
 '2006': 37.0,
 '2007': 4699.0,
 '2008': 9.0,
 '2009': 4514.0,
 '2010': 3253.0,
 '2011': 6454.0,
 '2013': 5019.0,
 '2014': 5436.0,
 '2016': 1221.0,
 '2017': 1805.0}

In [97]:
if 'Action' in archivo['genres'].unique():
    print('Si es posible')
else:
    print('No es posible la comparación')

Si es posible


## Reviews

In [3]:
with open('../Dataset/user_reviews.json/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    rows = [literal_eval(row) for row in file.readlines()]
reviews = pd.DataFrame(rows)
reviews = reviews.drop(columns=["user_url"])                    # El link de la review no tiene relevancia para este caso
#reviews = reviews.dropna(axis=0,how="any",subset=['reviews','user_id']) usado en caso de que existieran valores nulos, se comprobó que no es así
reviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
lista_reviews = []
for i in reviews['reviews']:
    lista_reviews.append(i)
    
reviews_individual = pd.DataFrame(lista_reviews[0])
reviews_individual['user_id'] = reviews['user_id'].head(1)

lista2_reviews = lista_reviews[1:]
for e in lista2_reviews:
    aux = pd.DataFrame(e)
    reviews_individual = pd.concat((reviews_individual,aux),ignore_index=True)

reviews_individual

KeyboardInterrupt: 

In [ ]:
def tokenizeText(text:str):
    '''Toma una cadena de texto para filtrar las palabras, si existen en ingles, además si son de un lenguaje las cuales sean útiles para lo que se necesita'''
    tokens = wt(text.lower())
    f_tokens = [token for token in tokens if token not in sw.words('english')]
    lemmatizer = wnl()
    l_tokens = [lemmatizer.lemmatize(token) for token in f_tokens]
    p_text = ' '.join(l_tokens)
    return p_text

reviews_individual['review'] = reviews_individual['review'].apply(tokenizeText)
reviews_individual

In [ ]:
analyzer = sia()
def get_sentiment(text:str):
    score = analyzer.polarity_scores(text)
    
    sentiment = 0
    if score['pos'] > score['neg']:
        if score['pos'] > score['neu']:
            sentiment = 2
        else:
            sentiment = 1
    else:
        if score['neg'] < score['neu']:
            sentiment = 1
    return sentiment
reviews_individual['sentiment'] = reviews_individual['review'].apply(get_sentiment)
reviews_individual

In [ ]:
reviews_individual = reviews_individual.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'review'])
reviews_individual

In [7]:
reviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25793,JustMielThings,"[{'funny': '', 'posted': 'Posted May 20.', 'la..."
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
